In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from configurationGM import gkey

In [10]:
#store csv into dataframe
WeatherData=pd.read_csv('./Resources/API_Weather_Data')
WeatherData.dropna()
WeatherData.head()

,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Latitude(deg.),Longitude(deg.)
0,-9.13,95,0,2.35,53.13,132.90
1,55.00,82,91,3.00,-46.60,168.33
2,27.30,58,75,17.90,59.70,30.79
3,78.19,74,34,24.16,-21.36,55.77
4,55.00,82,69,3.00,-46.19,168.86


In [7]:
#Humidity Heatmap: configure heat gmaps,
#use lat and long locations and humidity as the weight
#add Heatmap layer to map
Locations=WeatherData[['Latitude(deg.)','Longitude(deg.)']]
humidity = WeatherData["Humidity(%)"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(Locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
#create new dataframe fitting weather criteria.

WeatherData.head()
WeatherData.dropna()
Humidi=WeatherData.loc[WeatherData['Humidity(%)']<=60]
Wind = Humidi.loc[Humidi["Wind Speed (mph)"] <=15]
Cloud=Wind.loc[Wind['Cloudiness(%)']<=10]
MaxTemp=Cloud.loc[Cloud['Temperature(F)']<=85]
hotel_df=MaxTemp.loc[MaxTemp['Temperature(F)']>=75]
hotel_df['Hotel Name']=""
hotel_df['City']=""
hotel_df['Country']=""
hotel_df.dropna()
hotel_df


C:\Users\Andrew McGraw\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Andrew McGraw\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Andrew McGraw\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Latitude(deg.),Longitude(deg.),Hotel Name,City,Country
19,83.21,8,0,3.00,19.75,10.25,,,
166,81.77,32,5,3.02,-24.79,-50.01,,,
280,77.09,19,0,11.54,21.43,39.83,,,
316,83.75,58,2,9.80,-33.70,-53.46,,,
553,84.20,35,0,9.17,20.93,-17.03,,,
564,75.11,25,1,2.57,19.30,97.97,,,


In [24]:
#store into variable named hotel_df. add hotel name column
#set parameters for hotel search within 5000 meters
#hit google places api for each city's coordinates
#store the first hotel result into the dataframe
#plot markers on top of heatmap
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
locs=hotel_df[['Latitude(deg.)','Longitude(deg.)']]
print(locs)
hotels=[]
params = {
        
        "radius": '5000',
        "type": "hotel",
        "keyword": "hotel",
        "key": gkey,}
    
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["Latitude(deg.)"]
    long=row["Longitude(deg.)"]
    params['location']=f'{lat},{long}'

    # assemble url and make API request
        #print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
        
    # extract results
    results = response['results']
    try:
        hotels.append(response['results'][0]['name'])
    except IndexError:
        hotels.append(np.nan)
            
        #hotel_df.loc[index,"Hotel Name"]=results[1]['name']
        #hotel_df.loc[index,'City']=results[0]['name']
        #hotel_df.loc[index,'Country']=results[0]['plus_code'][0]['compound_code'][2]
        #hotel_df.loc[index,'Country']=results[0]['rating']
        #print(results)
hotels


     Latitude(deg.)  Longitude(deg.)
19            19.75            10.25
166          -24.79           -50.01
280           21.43            39.83
316          -33.70           -53.46
553           20.93           -17.03
564           19.30            97.97


[nan,
 'Hotel Buganville Express',
 'Pullman Zamzam Makkah',
 'Bertelli Chuí Hotel',
 'El Medina',
 'The Imperial Mae Hong Son Resort']

In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude(deg.)", "Longitude(deg.)"]]


#fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=temp, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))